<a href="https://colab.research.google.com/github/Fabiorsi1979/DS-Comunidade/blob/main/SQL_intermediario_subqueries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SUBQUERIES

### As subqueries são consultas dentro de outra consulta e podem utilizadas dentro das clausulas SELECT, FROM e WHERE.
#### - SELECT - como uma nova coluna de consulta
#### - FROM  - como filtro de uma consulta
#### - WHERE - como fonte de dados da coluna principal

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
path = '/content/drive/MyDrive/db_olist.sqlite'

In [5]:
import sqlite3
import pandas as pd

In [6]:
def subqueries_sql (*args):
  conexao = sqlite3.connect(path)
  cursor = conexao.cursor()

  cursor.execute(consulta)
  dados = cursor.fetchall()

  cursor.close()
  conexao.close()

  consulta_sql = pd.DataFrame(data= dados)
  return consulta_sql

#### 1. Qual o número de pedidos com o tipo de pagamento igual a “boleto”?


In [5]:
consulta = """
SELECT
    COUNT(o.order_id) AS Pedidos
FROM orders o
WHERE o.order_id IN (SELECT DISTINCT op.order_id FROM order_payments op WHERE op.payment_type = 'boleto')
"""
resultado =  subqueries_sql(consulta)
resultado.columns = ["Total de Pedidos"]
resultado

,Total de Pedidos
0,19784


#### 2. Refaça o exercício 01 usando união de tabelas.


In [6]:
consulta = """
SELECT
    COUNT(o.order_id) AS Pedidos
FROM orders o LEFT JOIN order_payments op ON ( op.order_id = o.order_id )
WHERE op.payment_type = 'boleto'
"""
resultado =  subqueries_sql(consulta)
resultado.columns = ["Total de Pedidos"]
resultado

,Total de Pedidos
0,19784


#### 3. Crie uma tabela que mostre a média de avaliações por dia, a média de preço por dia, a soma dos preços por dia, o preço mínimo por dia, o número de pedidos por dia e o número de clientes únicos que compraram no dia.


In [7]:
consulta = """
SELECT
    DATE(orr.review_creation_date) AS Dia_review,
    COUNT (orr.order_id) AS Total_Pedidos,
    AVG(orr.review_score) AS Avg_Aval
FROM order_reviews orr
GROUP BY DATE (orr.review_creation_date)
"""
t1 = subqueries_sql(consulta)
t1.columns=["Dia_review","Total Pedidos","Media Score"]
t1.head(4)

,Dia_review,Total Pedidos,Media Score
0,2016-10-02,1,1.0
1,2016-10-06,1,1.0
2,2016-10-09,1,1.0
3,2016-10-15,2,4.0


In [8]:
consulta = """
SELECT
    DATE(oi.shipping_limit_date) AS Dia_items,
    SUM (oi.price) AS Soma_price,
    MIN (oi.price) AS MIN_price,
    AVG (oi.price) AS Avg_price
FROM order_items oi
GROUP BY DATE(oi.shipping_limit_date)
"""
t2 = subqueries_sql(consulta)
t2.columns=["Dia_items","Valor total","Menor Venda","Ticket Médio"]
t2.head(4)

,Dia_items,Valor total,Menor Venda,Ticket Médio
0,2016-09-19,194.47,44.99,48.617500
1,2016-10-08,908.29,9.90,100.921111
2,2016-10-09,1545.57,22.37,128.797500
3,2016-10-10,7607.91,10.00,122.708226


In [9]:
consulta = """
SELECT
    DATE(O.order_approved_at) AS Dia_orders,
    COUNT(o.order_id) AS Contador_pedidos,
    COUNT(o.customer_id) AS Contador_clientes
FROM orders o
GROUP BY DATE(O.order_approved_at)
"""
t3 = subqueries_sql(consulta)
t3.columns=["Dia_order","Qtde Pedidos","Qtde Clientes"]
t3.head(4)

,Dia_order,Qtde Pedidos,Qtde Clientes
0,None,160,160
1,2016-09-15,1,1
2,2016-10-04,20,20
3,2016-10-05,11,11


#### Esquema para responder a questão 3 (SUBQUERIES)
~~~ Python
SELECT
    T3.Dia_orders,
    T1.Avg_Aval,
    T2.Avg_price,
    T2.Sum_price,
    T2.Min_price,
    T3.Contador_pedidos,
    T3.Contador_clientes
FROM (T3) AS T3 LEFT JOIN (T2) AS T2 ON ( T2.Dia_items = T3.Dia_orders )
                 LEFT JOIN (T1) AS T1 ON ( T1.Dia_review = T3.Dia_orders)
GROUP BY T3.Dia_orders
~~~

In [10]:
consulta = """
SELECT
    T3.Dia_orders,
    T1.Avg_Aval,
    T2.Avg_price,
    T2.Soma_price,
    T2.Min_price,
    T3.Contador_pedidos,
    T3.Contador_clientes
FROM (SELECT
        DATE(O.order_approved_at) AS Dia_orders,
        COUNT(o.order_id) AS Contador_pedidos,
        COUNT(o.customer_id) AS Contador_clientes
      FROM orders o
      GROUP BY DATE(O.order_approved_at)) AS T3 LEFT JOIN (SELECT
                                                              DATE(oi.shipping_limit_date) AS Dia_items,
                                                              SUM (oi.price) AS Soma_price,
                                                              MIN (oi.price) AS MIN_price,
                                                              AVG (oi.price) AS Avg_price
                                                           FROM order_items oi
                                                           GROUP BY DATE(oi.shipping_limit_date)) AS T2    ON ( T2.Dia_items = T3.Dia_orders )
                                                LEFT JOIN (SELECT
                                                              DATE(orr.review_creation_date) AS Dia_review,
                                                              COUNT (orr.order_id) AS Total_Pedidos,
                                                              AVG(orr.review_score) AS Avg_Aval
                                                           FROM order_reviews orr
                                                           GROUP BY DATE (orr.review_creation_date)) AS T1 ON ( T1.Dia_review = T3.Dia_orders)
WHERE T2.Soma_price NOT NULL
GROUP BY T3.Dia_orders
"""
resultado = subqueries_sql(consulta)
resultado.columns=["Data","Media Avaliações","Ticket Médio","Vendas Totais","Menor Venda","Pedidos Efetuados","Clientes Atendidos"]
resultado.head(5)

,Data,Media Avaliações,Ticket Médio,Vendas Totais,Menor Venda,Pedidos Efetuados,Clientes Atendidos
0,2016-10-08,NaN,100.921111,908.29,9.90,26,26
1,2016-10-09,1.0,128.797500,1545.57,22.37,10,10
2,2016-10-10,NaN,122.708226,7607.91,10.00,45,45
3,2016-10-11,NaN,141.347308,7350.06,18.90,28,28
4,2016-10-12,NaN,166.461613,5160.31,18.90,5,5


#### 4. Eu gostaria de saber, por categoria, a quantidade de produtos, o tamanho médio do produto, o tamanho médio da categoria alimentos e o tamanho médio geral.

In [19]:
consulta = """
SELECT
   p.product_category_name,
   COUNT(p.product_id) AS Total_produtos,
   AVG (product_length_cm) AS Tamanho_medio_produto,
   (SELECT AVG(p2.product_length_cm) FROM products p2 ) AS Tamanho_Med_Geral
FROM products p
GROUP BY p.product_category_name
"""
resultado = subqueries_sql(consulta)
resultado.columns=["Categorias","Total de Produtos","Tamanho Médio Produtos","Tamanho Médio Geral"]
print("-------------------------------------------------------------------------------------")
print(resultado.loc[resultado["Categorias"]=='alimentos'])
print("-------------------------------------------------------------------------------------")
resultado.head(4)

-------------------------------------------------------------------------------------
  Categorias  Total de Produtos  Tamanho Médio Produtos  Tamanho Médio Geral
2  alimentos                 82               21.292683            30.815078
-------------------------------------------------------------------------------------


,Categorias,Total de Produtos,Tamanho Médio Produtos,Tamanho Médio Geral
0,None,610,28.719212,30.815078
1,agro_industria_e_comercio,74,36.527027,30.815078
2,alimentos,82,21.292683,30.815078
3,alimentos_bebidas,104,23.403846,30.815078


#### 5. . Qual o nome da categoria de produto com o maior preço de venda?
####         **Encontre a categoria do produto somente com Subqueries.**

In [18]:
consulta = """
SELECT
    oi.product_id,
     MAX(oi.price)
FROM order_items oi
"""
t1 = subqueries_sql(consulta)
t1.columns=["Codigo Produto","Maior Venda"]
t1

,Codigo Produto,Maior Venda
0,489ae2aa008f021502940f251d4cce7f,6735.0


In [20]:
consulta = """
SELECT
    t1.product_id
FROM (SELECT
          oi.product_id,
          MAX(oi.price)
          FROM order_items oi) AS T1
"""
t2 = subqueries_sql(consulta)
t2.columns=["Codigo Produto"]
t2

,Codigo Produto
0,489ae2aa008f021502940f251d4cce7f


In [21]:
consulta = """
SELECT
    p.product_category_name
FROM products p
WHERE p.product_id = (SELECT
                          t1.product_id
                      FROM (SELECT
                                oi.product_id,
                                MAX(oi.price)
                                FROM order_items oi) AS T1)
"""
resultado = subqueries_sql(consulta)
resultado.columns=["Categoria com maior preço de Venda"]
resultado

,Categoria com maior preço de Venda
0,utilidades_domesticas
